In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os

import numpy as np
import pandas as pd

import kraft

In [ ]:
directory_path = os.path.expanduser("~/Downloads/")

## score_sample_and_set

In [ ]:
for n_element in (10,):

    element_scores = np.random.normal(size=n_element // 2)

    element_scores = pd.Series(
        data=np.sort(np.concatenate((-element_scores, element_scores))),
        index=("Element {}".format(i) for i in range(n_element)),
    )

    element_scores.index.name = "Element"

    element_scores.name = "Score"

    for n_skip in (1,):

        if n_skip < 1:

            n_skip = int(n_skip * n_element)

        for n_top in (2,):

            if n_top < 1:

                n_top = int(n_top * n_element)

            for rate_top_hit in (1,):

                if rate_top_hit < 1:

                    rate_top_hit = int(rate_top_hit * n_top)

                for side in ("+",):

                    set_to_elements = []

                    elements = element_scores.index.to_numpy()

                    if "-" in side:

                        set_to_elements += elements[
                            n_skip : n_skip + n_top : rate_top_hit
                        ].tolist()

                    if "+" in side:

                        set_to_elements += elements[
                            -(n_skip + 1) : -(n_skip + 1 + n_top) : -rate_top_hit
                        ].tolist()

                    for method in ("classic", "2020"):

                        kraft.sea.score_sample_and_set(
                            element_scores,
                            set_to_elements,
                            method=method,
                            plot_process=True,
                        )

In [ ]:
element_scores = pd.read_csv(
    "data/gene_score.tsv.gz", sep="\t", index_col=0, squeeze=True
)

set_to_elements = kraft.gmt.read("data/c2.all.v7.1.symbols.gmt")

In [ ]:
method = "2020"

In [ ]:
sets = ["COLLER_MYC_TARGETS_UP"]

sets += list(np.random.choice(list(set_to_elements.keys()), size=2, replace=False))

for set_ in sets:

    kraft.sea.score_sample_and_set(
        element_scores, set_to_elements[set_], method=method, title=set_
    )

In [ ]:
set_score = {}

for set_, elements in set_to_elements.items():

    set_score[set_] = kraft.sea.score_sample_and_set(
        element_scores, elements, method=method, plot=False, title=set_
    )

set_score = pd.Series(data=set_score).sort_values()

n_extreme = 2

for i in list(range(n_extreme)) + list(range(-n_extreme, 0)):

    set_ = set_score.index.to_numpy()[i]

    kraft.sea.score_sample_and_set(
        element_scores, set_to_elements[set_], method=method, title=set_
    )

## score_samples_and_sets

In [ ]:
element_scores = pd.read_csv(
    "data/gene_score.tsv.gz", sep="\t", index_col=0, squeeze=True
)

element_x_sample = pd.concat(
    (element_scores, element_scores * 10, element_scores / 10), axis=1
)

element_x_sample.columns = (
    "Sample {}".format(i) for i in range(element_x_sample.shape[1])
)

element_x_sample

In [ ]:
set_to_elements = kraft.gmt.read("data/h.all.v7.1.symbols.gmt")

In [ ]:
set_x_sample = kraft.sea.score_samples_and_sets(
    element_x_sample, set_to_elements, n_job=4, directory_path=directory_path,
)

set_x_sample